In [183]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple
import numpy.ma as ma

# Data Preparation

In [137]:
cols,cols1,cols2,cols3=['sentence'], ['Vocabulary'],['slot'],['intent']
df=pd.read_csv("train.csv",names=cols)
df1=pd.read_csv("vocab.csv",names=cols1)
df2=pd.read_csv("train_slots.csv",names=cols2)
df3=pd.read_csv("slots.csv",names=cols3)
df4=pd.read_csv("test.csv",names=cols)
intent=pd.read_csv("intents.csv",names=cols3)

In [138]:
print(intent[0:5],"\n\n",df2.head(),"\n\n",df1.head())

                      intent
0               abbreviation
1                   aircraft
2  aircraft+flight+flight_no
3                    airfare
4             airfare+flight 

                                                 slot
0  128 128 128 128 128 128 48 128 35 100 128 128 ...
1    128 128 128 128 128 128 48 128 78 128 26 33 128
2  128 128 128 128 45 108 128 48 110 128 128 35 1...
3                       128 21 128 128 48 128 78 128
4         128 66 119 128 128 48 128 78 21 38 103 128 

   Vocabulary
0         'd
1      'hare
2        'll
3         'm
4        're


In [139]:
train=[]
test=[]
slot1=[]
for i in range(df.shape[0]):
    sntce=df.sentence[i].split()
    slot=df2.slot[i].split()
    array1=[]
    array2=[]
    
    for j in range(len(sntce)):
        array1.append(df1.Vocabulary[int(sntce[j])])
    train.append(" ".join(array1))
    
    
    for k in range(len(slot)):
        array2.append(df3.intent[int(slot[k])])
    slot1.append(" ".join(array2))
    
for i1 in range(df4.shape[0]):
    sntce1=df4.sentence[i1].split()
    array3=[]
    for j1 in range(len(sntce1)):
        array3.append(df1.Vocabulary[int(sntce1[j1])])
    test.append(" ".join(array3))
    
    
   

In [140]:
train = [t[:-1] for t in train]
train = [t.split(" ") for t in train]
train = [t[1:-1] for t in train]

test = [t[:-1] for t in test]
test = [t.split(" ") for t in test]
test = [t[1:-1] for t in test]

slot1 = [t[:-1] for t in slot1]
slot1 = [t.split(" ") for t in slot1]
slot1 = [t[1:-1] for t in slot1]

intent = [t for t in intent['intent']]


In [141]:
print(intent[0:5],"\n\n",train[:5],"\n\n",slot1[:5])

['abbreviation', 'aircraft', 'aircraft+flight+flight_no', 'airfare', 'airfare+flight'] 

 [['i', 'want', 'to', 'fly', 'from', 'boston', 'at', '838', 'am', 'and', 'arrive', 'in', 'denver', 'at', '1110', 'in', 'the', 'morning'], ['what', 'flights', 'are', 'available', 'from', 'pittsburgh', 'to', 'baltimore', 'on', 'thursday', 'morning'], ['what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '755', 'am', 'flight', 'leaving', 'washington'], ['cheapest', 'airfare', 'from', 'tacoma', 'to', 'orlando'], ['round', 'trip', 'fares', 'from', 'pittsburgh', 'to', 'philadelphia', 'under', '1000', 'dollars']] 

 [['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day'], ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', 'O', 'B-depart_date.day_name', 'B-depart_time.period_of_day'], ['O', 'O', 'O', 'B-flight_t

# Tokenization with special tokens in seq2seq:



In [142]:
flatten = lambda l: [item for sublist in l for item in sublist]
vocab = set(flatten(train))
slot_tag = set(flatten(slot1))
intent_tag = set(intent)

In [143]:
print(train[:5],"\n\n", slot_tag, "\n\n" ,intent_tag)

[['i', 'want', 'to', 'fly', 'from', 'boston', 'at', '838', 'am', 'and', 'arrive', 'in', 'denver', 'at', '1110', 'in', 'the', 'morning'], ['what', 'flights', 'are', 'available', 'from', 'pittsburgh', 'to', 'baltimore', 'on', 'thursday', 'morning'], ['what', 'is', 'the', 'arrival', 'time', 'in', 'san', 'francisco', 'for', 'the', '755', 'am', 'flight', 'leaving', 'washington'], ['cheapest', 'airfare', 'from', 'tacoma', 'to', 'orlando'], ['round', 'trip', 'fares', 'from', 'pittsburgh', 'to', 'philadelphia', 'under', '1000', 'dollars']] 

 {'I-depart_date.today_relative', 'B-meal_code', 'I-airport_name', 'B-stoploc.city_name', 'B-flight_number', 'I-stoploc.city_name', 'B-depart_date.today_relative', 'I-toloc.city_name', 'B-return_date.date_relative', 'B-airline_code', 'I-fare_basis_code', 'B-fromloc.airport_code', 'I-flight_stop', 'B-arrive_date.today_relative', 'B-toloc.airport_name', 'B-return_date.today_relative', 'B-arrive_time.time_relative', 'I-flight_time', 'B-fromloc.airport_name', 

In [144]:
LENGTH=50 # All the sequences in your batch should have the same length
sin=[]
sout=[]
sin_test=[]
sout_test=[]

In [145]:
for i in range(len(train)):
    temp = train[i]
    if len(temp)<LENGTH:
        temp.append('<EOS>')
        while len(temp)<LENGTH:
            temp.append('<PAD>')
    else:
        temp = temp[:LENGTH]
        temp[-1]='<EOS>'
    sin.append(temp)
    
    temp = slot1[i]
    if len(temp)<LENGTH:
        while len(temp)<LENGTH:
            temp.append('<PAD>')
    else:
        temp = temp[:LENGTH]
        temp[-1]='<EOS>'
    sout.append(temp)



In [146]:
for i in range(len(test)):
    temp = test[i]
    if len(temp)<LENGTH:
        temp.append('<EOS>')
        while len(temp)<LENGTH:
            temp.append('<PAD>')
    else:
        temp = temp[:LENGTH]
        temp[-1]='<EOS>'
    sin_test.append(temp)
    
    temp = slot1[i]
    if len(temp)<LENGTH:
        while len(temp)<LENGTH:
            temp.append('<PAD>')
    else:
        temp = temp[:LENGTH]
        temp[-1]='<EOS>'
    sout_test.append(temp)

In [147]:
train = list(zip(sin,sout,intent))
print(train[0])

(['i', 'want', 'to', 'fly', 'from', 'boston', 'at', '838', 'am', 'and', 'arrive', 'in', 'denver', 'at', '1110', 'in', 'the', 'morning', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], 'abbreviation')


In [148]:
test = list(zip(sin_test,sout_test,intent))
print(test[0])

(['i', 'would', 'like', 'to', 'find', 'a', 'flight', 'from', 'charlotte', 'to', 'las', 'vegas', 'that', 'makes', 'a', 'stop', 'in', 'st.', 'louis', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>'], ['O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-depart_time.time', 'I-depart_time.time', 'O', 'O', 'O', 'B-toloc.city_name', 'O', 'B-arrive_time.time', 'O', 'O', 'B-arrive_time.period_of_day', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<EOS>'], 'abbreviation')


# Dictionaries of words and tags.

In [149]:
def get_info_from_training_data(data):
    seq_in, seq_out, intent = list(zip(*data))
    vocab = set(flatten(seq_in))
    slot_tag = set(flatten(seq_out))
    intent_tag = set(intent)
   #word2index
    word2index = {'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3}
    for token in vocab:
        if token not in word2index.keys():
            word2index[token] = len(word2index)

    # index2word
    index2word = {v: k for k, v in word2index.items()}

    # tag2index
    tag2index = {'<PAD>': 0, '<UNK>': 1, "O": 2}
    for tag in slot_tag:
        if tag not in tag2index.keys():
            tag2index[tag] = len(tag2index)

    # index2tag
    index2tag = {v: k for k, v in tag2index.items()}

    # intent2index
    intent2index = {'<UNK>': 0}
    for ii in intent_tag:
        if ii not in intent2index.keys():
            intent2index[ii] = len(intent2index)

    # index2intent
    index2intent = {v: k for k, v in intent2index.items()}
    return word2index, index2word, tag2index, index2tag, intent2index, index2intent


word2index, index2word, slot2index, index2slot, intent2index, index2intent = get_info_from_training_data(train)


In [150]:
index_seq2slot = lambda s, index2slot: [index2slot[i] for i in s]
index_seq2word = lambda s, index2word: [index2word[i] for i in s]


In [151]:
print(word2index,"\n\n",index2word,"\n\n",slot2index)

{'<PAD>': 0, '<UNK>': 1, '<SOS>': 2, '<EOS>': 3, 'fares': 4, 'would': 5, 'columbus': 6, 'newark': 7, 'at': 8, 'after': 9, 'under': 10, 'arrive': 11, 'all': 12, '7': 13, 'bwi': 14, 'baltimore': 15, 'trip': 16, 'houston': 17, 'by': 18, 'angeles': 19, 'class': 20, 'want': 21, 'and': 22, 'show': 23, 'francisco': 24, 'information': 25, 'thursday': 26, 'cleveland': 27, 'first': 28, 'have': 29, 'airlines': 30, 'boston': 31, 'need': 32, 'cheapest': 33, '755': 34, 'washington': 35, 'united': 36, 'please': 37, 'los': 38, 'tomorrow': 39, 'round': 40, 'ground': 41, 'way': 42, 'philadelphia': 43, 'me': 44, 'is': 45, "'d": 46, 'pm': 47, 'fly': 48, 'dollars': 49, 'am': 50, 'flight': 51, 'for': 52, 'kind': 53, 'planes': 54, "'s": 55, 'denver': 56, '1110': 57, 'what': 58, 'a': 59, 'atlanta': 60, 'flights': 61, 'pittsburgh': 62, 'before': 63, 'list': 64, '838': 65, 'morning': 66, 'diego': 67, 'airport': 68, 'tacoma': 69, 'next': 70, 'aircraft': 71, 'kinds': 72, 'give': 73, 'leaving': 74, 'some': 75, 'bo

# Map the sentences to a sequence of numbers.

In [152]:
def to_index(train, word2index, slot2index, intent2index):
    new_train = []
    for sin, sout, intent in train:
        sin_ix = list(map(lambda i: word2index[i] if i in word2index else word2index["<UNK>"],
                          sin))
        true_length = sin.index("<EOS>")
        sout_ix = list(map(lambda i: slot2index[i] if i in slot2index else slot2index["<UNK>"],
                           sout))
        intent_ix = intent2index[intent] if intent in intent2index else intent2index["<UNK>"]
        new_train.append([sin_ix, true_length, sout_ix, intent_ix])
    return new_train


In [153]:
index_train = to_index(train, word2index, slot2index, intent2index)
index_test = to_index(test, word2index, slot2index, intent2index)
print(index_train[0])

[[86, 21, 84, 48, 94, 31, 8, 65, 50, 22, 11, 80, 56, 8, 57, 80, 85, 66, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 18, [2, 2, 2, 2, 2, 22, 2, 13, 14, 2, 2, 2, 9, 2, 24, 2, 2, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 24]



# RNN- LSTM network with an embedding layer

In [154]:
input_steps = 50
embedding_size = 64
hidden_size = 100
n_layers = 2
batch_size = 16
vocab_size = 871
slot_size = 122
intent_size = 22
epoch_num = 5


In [155]:
encoder_inputs = tf.placeholder(tf.int32, [input_steps, batch_size],
                                             name='encoder_inputs')
# Actual length of each sentence input，apart from padding
encoder_inputs_actual_length = tf.placeholder(tf.int32, [batch_size],
                                                   name='encoder_inputs_actual_length')
decoder_targets = tf.placeholder(tf.int32, [batch_size, input_steps],
                                      name='decoder_targets')
intent_targets = tf.placeholder(tf.int32, [batch_size],
                                     name='intent_targets')

In [156]:
embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size],
                                                        -0.1, 0.1), dtype=tf.float32, name="embedding")

encoder_inputs_embedded = tf.nn.embedding_lookup(embeddings, encoder_inputs)

In [157]:
encoder_inputs_embedded

<tf.Tensor 'embedding_lookup_2/Identity:0' shape=(50, 16, 64) dtype=float32>

# Define the RNN-LSTM Cell model

In [158]:
encoder_f_cell = LSTMCell(hidden_size)
encoder_b_cell = LSTMCell(hidden_size)

# Encoding seq_2_seq 

In [159]:
(encoder_fw_outputs, encoder_bw_outputs), (encoder_fw_final_state, encoder_bw_final_state) = \
    tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_f_cell,
                                    cell_bw=encoder_b_cell,
                                    inputs=encoder_inputs_embedded,
                                    sequence_length=encoder_inputs_actual_length,
                                    dtype=tf.float32, time_major=True)
encoder_outputs = tf.concat((encoder_fw_outputs, encoder_bw_outputs), 2)

encoder_final_state_c = tf.concat(
    (encoder_fw_final_state.c, encoder_bw_final_state.c), 1)

encoder_final_state_h = tf.concat(
    (encoder_fw_final_state.h, encoder_bw_final_state.h), 1)

encoder_final_state = LSTMStateTuple(
    c=encoder_final_state_c,
    h=encoder_final_state_h
)

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


In [160]:
print("encoder_outputs: ", encoder_outputs)
print("encoder_outputs[0]: ", encoder_outputs[0])
print("encoder_final_state_c: ", encoder_final_state_c)


encoder_outputs:  Tensor("concat:0", shape=(50, 16, 200), dtype=float32)
encoder_outputs[0]:  Tensor("strided_slice:0", shape=(16, 200), dtype=float32)
encoder_final_state_c:  Tensor("concat_1:0", shape=(16, 200), dtype=float32)


In [161]:
decoder_lengths = encoder_inputs_actual_length

In [162]:
slot_W = tf.Variable(tf.random_uniform([hidden_size * 2, slot_size], -1, 1),
                             dtype=tf.float32, name="slot_W")
slot_b = tf.Variable(tf.zeros([slot_size]), dtype=tf.float32, name="slot_b")
intent_W = tf.Variable(tf.random_uniform([hidden_size * 2, intent_size], -0.1, 0.1),
                               dtype=tf.float32, name="intent_W")
intent_b = tf.Variable(tf.zeros([intent_size]), dtype=tf.float32, name="intent_b")

In [163]:
intent_logits = tf.add(tf.matmul(encoder_final_state_h, intent_W), intent_b)
intent = tf.argmax(intent_logits, axis=1)

In [164]:
sos_time_slice = tf.ones([batch_size], dtype=tf.int32, name='SOS') * 2
sos_step_embedded = tf.nn.embedding_lookup(embeddings, sos_time_slice)
pad_step_embedded = tf.zeros([batch_size, hidden_size * 2 + embedding_size],
                             dtype=tf.float32)

In [165]:
def initial_fn():
    initial_elements_finished = (0 >= decoder_lengths)  # all False at the initial step
    initial_input = tf.concat((sos_step_embedded, encoder_outputs[0]), 1)
    return initial_elements_finished, initial_input

In [166]:
def sample_fn(time, outputs, state):
    prediction_id = tf.to_int32(tf.argmax(outputs, axis=1))
    return prediction_id

In [167]:
def next_inputs_fn(time, outputs, state, sample_ids):
    # Output category on the previous time node，obtain embedding again as input to the next time node
    pred_embedding = tf.nn.embedding_lookup(embeddings, sample_ids)
    next_input = tf.concat((pred_embedding, encoder_outputs[time]), 1)
    elements_finished = (time >= decoder_lengths)  # this operation produces boolean tensor of [batch_size]
    all_finished = tf.reduce_all(elements_finished)  # -> boolean scalar
    next_inputs = tf.cond(all_finished, lambda: pad_step_embedded, lambda: next_input)
    next_state = state
    return elements_finished, next_inputs, next_state

In [168]:
my_helper = tf.contrib.seq2seq.CustomHelper(initial_fn, sample_fn, next_inputs_fn)

# Define the Decoding

In [169]:
def decode(helper, scope, reuse=None):
    with tf.variable_scope(scope, reuse=reuse):
        memory = tf.transpose(encoder_outputs, [1, 0, 2])
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
            num_units=hidden_size, memory=memory,
            memory_sequence_length=encoder_inputs_actual_length)
         #Loss and optimizer
        cell = tf.contrib.rnn.LSTMCell(num_units=hidden_size * 2)
        attn_cell = tf.contrib.seq2seq.AttentionWrapper(
            cell, attention_mechanism, attention_layer_size=hidden_size)
        out_cell = tf.contrib.rnn.OutputProjectionWrapper(
            attn_cell, slot_size, reuse=reuse
        )
        decoder = tf.contrib.seq2seq.BasicDecoder(
            cell=out_cell, helper=helper,
            initial_state=out_cell.zero_state(
                dtype=tf.float32, batch_size=batch_size))
        # initial_state=encoder_final_state)
        final_outputs, final_state, final_sequence_lengths = tf.contrib.seq2seq.dynamic_decode(
            decoder=decoder, output_time_major=True,
            impute_finished=True, maximum_iterations=input_steps
        )
        return final_outputs

In [170]:
outputs = decode(my_helper, 'decode')
print("outputs: ", outputs)
print("outputs.rnn_output: ", outputs.rnn_output)
print("outputs.sample_id: ", outputs.sample_id)

outputs:  BasicDecoderOutput(rnn_output=<tf.Tensor 'decode/decoder/TensorArrayStack/TensorArrayGatherV3:0' shape=(?, 16, 122) dtype=float32>, sample_id=<tf.Tensor 'decode/decoder/TensorArrayStack_1/TensorArrayGatherV3:0' shape=(?, 16) dtype=int32>)
outputs.rnn_output:  Tensor("decode/decoder/TensorArrayStack/TensorArrayGatherV3:0", shape=(?, 16, 122), dtype=float32)
outputs.sample_id:  Tensor("decode/decoder/TensorArrayStack_1/TensorArrayGatherV3:0", shape=(?, 16), dtype=int32)


In [171]:
decoder_prediction = outputs.sample_id
decoder_max_steps, decoder_batch_size, decoder_dim = tf.unstack(tf.shape(outputs.rnn_output))
decoder_targets_time_majored = tf.transpose(decoder_targets, [1, 0])
decoder_targets_true_length = decoder_targets_time_majored[:decoder_max_steps]
print("decoder_targets_true_length: ", decoder_targets_true_length)

decoder_targets_true_length:  Tensor("strided_slice_1:0", shape=(?, 16), dtype=int32)


In [172]:
mask = tf.to_float(tf.not_equal(decoder_targets_true_length, 0))

Instructions for updating:
Use tf.cast instead.


In [173]:
loss_slot = tf.contrib.seq2seq.sequence_loss(outputs.rnn_output, decoder_targets_true_length, weights=mask)

In [174]:
## Loss and optimizer
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
    labels=tf.one_hot(intent_targets, depth=intent_size, dtype=tf.float32),
    logits=intent_logits)
loss_intent = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [175]:
loss = loss_slot + loss_intent
optimizer = tf.train.AdamOptimizer(name="a_optimizer")
grads, vars = zip(*optimizer.compute_gradients(loss))
gradients, _ = tf.clip_by_global_norm(grads, 5)  # clip gradients
train_op = optimizer.apply_gradients(zip(grads, vars))

In [176]:
def step(sess, mode, trarin_batch):
    """ perform each batch"""
    if mode not in ['train', 'test']:
        print >> sys.stderr, 'mode is not supported'
        sys.exit(1)
    unziped = list(zip(*trarin_batch))
    if mode == 'train':
        output_feeds = [train_op, loss, decoder_prediction,
                        intent]
        feed_dict = {encoder_inputs: np.transpose(unziped[0], [1, 0]),
                     encoder_inputs_actual_length: unziped[1],
                     decoder_targets: unziped[2],
                     intent_targets: unziped[3]}
    if mode in ['test']:
        output_feeds = [decoder_prediction, intent]
        feed_dict = {encoder_inputs: np.transpose(unziped[0], [1, 0]),
                     encoder_inputs_actual_length: unziped[1]}

    results = sess.run(output_feeds, feed_dict=feed_dict)
    return results

In [177]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [178]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch

In [179]:
#Testing

In [180]:

def accuracy_score(true_data, pred_data, true_length=None):
    true_data = np.array(true_data)
    pred_data = np.array(pred_data)
    assert true_data.shape == pred_data.shape
    if true_length is not None:
        val_num = np.sum(true_length)
        assert val_num != 0
        res = 0
        for i in range(true_data.shape[0]):
            res += np.sum(true_data[i, :true_length[i]] == pred_data[i, :true_length[i]])
    else:
        val_num = np.prod(true_data.shape)
        assert val_num != 0
        res = np.sum(true_data == pred_data)
    res /= float(val_num)
    return res
def get_data_from_sequence_batch(true_batch, pred_batch, padding_token):
    """[[3,1,2,0,0,0],[5,2,1,4,0,0]] -> [3,1,2,5,2,1,4]"""
    true_ma = ma.masked_equal(true_batch, padding_token)
    pred_ma = ma.masked_array(pred_batch, true_ma.mask)
    true_ma = true_ma.flatten()
    pred_ma = pred_ma.flatten()
    true_ma = true_ma[~true_ma.mask]
    pred_ma = pred_ma[~pred_ma.mask]
    return true_ma, pred_ma


def f1_for_sequence_batch(true_batch, pred_batch, average="micro", padding_token=0):
    true, pred = get_data_from_sequence_batch(true_batch, pred_batch, padding_token)
    labels = list(set(true))
    return f1_score(true, pred, labels=labels, average=average)


def accuracy_for_sequence_batch(true_batch, pred_batch, padding_token=0):
    true, pred = get_data_from_sequence_batch(true_batch, pred_batch, padding_token)
    return accuracy_score(true, pred)


# Predicition and Slot Filling

In [182]:
for epoch in range(epoch_num):
    mean_loss = 0.0
    train_loss = 0.0
    for i, batch in enumerate(getBatch(batch_size, index_train)):
        _, loss_v, decoder_prediction_v, intent_v = step(sess, "train", batch)
        mean_loss += loss_v
        train_loss += loss_v
        if i % 30 == 0:
            if i > 0:
                mean_loss = mean_loss / 30.0
            print('Average train loss at epoch %d, step %d: %f' % (epoch, i, mean_loss))
            mean_loss = 0
    train_loss /= (i + 1)
    print("[Epoch {}] Average train loss: {}".format(epoch, train_loss))

    pred_slots = []
    for j, batch in enumerate(getBatch(batch_size, index_test)):
        decoder_prediction_v, intent_v = step(sess, "test", batch)
        decoder_prediction_v = np.transpose(decoder_prediction_v, [1, 0])
        if j == 0:
            index = random.choice(range(len(batch)))
            print("Input Sentence        : ", index_seq2word(batch[index][0], index2word))
            print("Slot Truth            : ", index_seq2slot(batch[index][2], index2slot))
            print("Slot Prediction       : ", index_seq2slot(decoder_prediction_v[index], index2slot))
            print("Intent Truth          : ", index2intent[batch[index][3]])
            print("Intent Prediction     : ", index2intent[intent_v[index]])
        slot_pred_length = list(np.shape(decoder_prediction_v))[1]
        pred_padded = np.lib.pad(decoder_prediction_v, ((0, 0), (0, input_steps-slot_pred_length)),
                                 mode="constant", constant_values=0)
        pred_slots.append(pred_padded)
        true_slot = np.array((list(zip(*batch))[2]))
        true_length = np.array((list(zip(*batch))[1]))
        true_slot = true_slot[:, :slot_pred_length]
        slot_acc = accuracy_score(true_slot, decoder_prediction_v, true_length)
        intent_acc = accuracy_score(list(zip(*batch))[3], intent_v)
        print("slot accuracy: {}, intent accuracy: {}".format(slot_acc, intent_acc))
    pred_slots_a = np.vstack(pred_slots)
    true_slots_a = np.array(list(zip(*index_test))[2])[:pred_slots_a.shape[0]]
    print("F1 score for epoch {}: {}\n".format(epoch, f1_for_sequence_batch(true_slots_a, pred_slots_a)))

Average train loss at epoch 0, step 0: 6.999569
[Epoch 0] Average train loss: 6.999568939208984
Input Sentence        :  ['<UNK>', 'morning', 'i', 'would', 'like', 'to', 'fly', 'from', 'columbus', 'to', '<UNK>', '<EOS>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
Slot Truth            :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-fromloc.city_name', 'O', 'B-toloc.city_name', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>